In [24]:
from utils import load_json
import pandas as pd 

In [ ]:
pct_chg = load_json("../database/nasdaq.json")
portfolio = load_json("../database/model_portfolio.json")

pct_chg = pd.DataFrame().from_dict(pct_chg)
pct_chg = pct_chg.pivot(index="date_f", columns="symbol", values="percentageChange")

def metrics(pnl):
    output = pd.Series()
    shifts = [1, 5, 21, 63, 126]
    for s in shifts:
        output[f"{s}d_sharpe"] = pnl.rolling(s).mean().item() / pnl.rolling(s).std().item()
        output[f"{s}d_vol"] = pnl.rolling(s).std().item()

    output["max_drawdown"] = (pnl.cummax() - pnl).max()
    return output

names = set(p["name"] for p in portfolio)

output = []
for n in names:
    w = {p["date"]: p["portfolio"] for p in portfolio if p["name"] == n}
    w = pd.DataFrame().from_dict(w).T
    pnl = (w * pct_chg).sum(axis=1)
    summary = metrics(pnl)
    summary["name"] = n
    output.append(summary.to_dict())

In [88]:
output

[{'1d_sharpe': nan,
  '1d_vol': nan,
  '5d_sharpe': nan,
  '5d_vol': nan,
  '21d_sharpe': nan,
  '21d_vol': nan,
  '63d_sharpe': nan,
  '63d_vol': nan,
  '126d_sharpe': nan,
  '126d_vol': nan,
  '252d_sharpe': nan,
  '252d_vol': nan,
  'max_drawdown': 0.0,
  'name': 'openai_fundamental'}]

1d_sharpe      NaN
1d_vol         NaN
5d_sharpe      NaN
5d_vol         NaN
21d_sharpe     NaN
21d_vol        NaN
63d_sharpe     NaN
63d_vol        NaN
126d_sharpe    NaN
126d_vol       NaN
252d_sharpe    NaN
252d_vol       NaN
dtype: object